# Week 3: Identify Risk Factors for Infection

<span style="color:red">
**UPDATE**

Thank you again for the previous analysis. We will next be publishing a public health advisory that warns of specific infection risk factors of which individuals should be aware. Please advise as to which population characteristics are associated with higher infection rates. 
</span>

Your goal for this notebook will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [1]:
import cudf
import cuml

## Load Data

Begin by loading the data you've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook you will need all columns of the data.

In [4]:
gdf = cudf.read_csv('./data/week3.csv')
gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


In [5]:
gdf.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 58479894 entries, 0 to 58479893
Data columns (total 4 columns):
 #   Column      Dtype
---  ------      -----
 0   age         int64
 1   sex         object
 2   employment  object
 3   infected    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.4+ GB


## Calculate Infection Rates by Employment Code

Convert the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [6]:
gdf['infected'] = gdf['infected'].astype('float32')
gdf.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 58479894 entries, 0 to 58479893
Data columns (total 4 columns):
 #   Column      Dtype
---  ------      -----
 0   age         int64
 1   sex         object
 2   employment  object
 3   infected    float32
dtypes: float32(1), int64(1), object(2)
memory usage: 1.2+ GB


In [9]:
gdf['infected'].value_counts()

0.0    58186187
1.0      293707
Name: infected, dtype: int32

Now, produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lower** than for type "X", even though more people within that employment type were infected.

In [10]:
gdf['employment'].value_counts()

U          12459115
V          10098466
Z           7161907
Q           3802602
G           3549465
P           3006149
C           2653753
M           2214336
F           2075628
O           1843446
R, S, T     1669197
I           1556575
H           1398342
N           1367137
J           1180372
K           1122406
B, D, E      486785
L            346470
A            305755
X            181988
Name: employment, dtype: int32

In [20]:
employ_type_rate = gdf.groupby('employment')['infected'].mean().reset_index()
employ_type_rate = employ_type_rate_list.sort_values('infected',ascending = False)
employ_type_rate

,employment,infected
12,Q,0.012756
13,I,0.010354
2,V,0.007590
17,P,0.006190
3,Z,0.005655
10,"R, S, T",0.005390
7,O,0.005284
14,L,0.004970
16,G,0.004948
19,N,0.004784


Finally, read in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.

In [19]:
code_guide = cudf.read_csv('./data/code_guide.csv')
code_guide

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"
5,H,Transport & storage
6,I,Accommodation & food services
7,J,Information & communication
8,K,Financial & insurance activities
9,L,Real estate activities


## Calculate Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Group by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [30]:
employmenttype_sex_rate = gdf.groupby(['employment','sex'])['infected'].mean().reset_index()
employmenttype_sex_rate = employmenttype_sex_rate.sort_values('infected',ascending = False)
employmenttype_sex_rate

,employment,sex,infected
28,I,f,0.015064
5,Q,f,0.014947
2,V,f,0.010852
18,"B, D, E",f,0.007973
13,"R, S, T",f,0.007748
8,O,f,0.007719
6,K,f,0.007672
4,M,f,0.007645
23,J,f,0.007645
25,C,f,0.007630


## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 3 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to continue work in other notebooks, please shutdown the kernel.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)